In [1]:
import geopandas as gpd
# import plotly.graph_objects as go
import folium
import json

In [2]:

data = gpd.read_file('Youssoufia.geojson')

In [3]:
import folium

map_center = [data.iloc[0]['geometry'].y, data.iloc[0]['geometry'].x]
map_zoom = 12
my_map = folium.Map(location=map_center, zoom_start=map_zoom, tiles='OpenStreetMap')

for index, row in data.iterrows():
    tags = row['tags']
    if 'amenity' in tags and tags['amenity'] == 'pharmacy':
        latitude, longitude = row['geometry'].y, row['geometry'].x
        tags_str = json.dumps(tags, indent=4)
        name = tags.get('name', 'Pharmacy')
        
        popup_content = f"<b>{name}<br>"
        folium.Marker([latitude, longitude], popup=popup_content, icon=folium.Icon(color='red')).add_to(my_map)

my_map


In [4]:
import json
with open('Specified_location_in_Youssofia.json', 'r') as file:
    data_json = json.load(file)

In [5]:
import folium

map_center = [32.248, -8.523]
my_map = folium.Map(location=map_center, zoom_start=15)

colors = {
    "School": "blue",
    "Coffee shop": "green",
}

for feature in data_json["features"]:
    name = feature["properties"]["name"]
    location_type = feature["properties"]["type"]
    lat, lon = feature["geometry"]["coordinates"]

    # Using default Folium icons
    icon = folium.Icon(color=colors.get(location_type, "red"))
    folium.Marker([lat, lon], popup=name, icon=icon).add_to(my_map)

my_map


In [17]:
from geopy.distance import geodesic

def calculer_distance(lat1, lon1, lat2, lon2):
    point1 = (lat1, lon1)
    point2 = (lat2, lon2)
    distance = geodesic(point1, point2).kilometers
    return distance

def display_distance_on_map(point1_name, point2_name, data_json):

    for feature in data_json["features"]:
        name = feature["properties"]["name"]
        if name == point1_name:
            youcode_coordinates = feature["geometry"]["coordinates"]
        elif name == point2_name:
            monte_carlo_coordinates = feature["geometry"]["coordinates"]

    point1_latitude, point1_longitude = youcode_coordinates
    point2_latitude, point2_longitude = monte_carlo_coordinates

    distance_entre_points = calculer_distance(point1_latitude, point1_longitude, point2_latitude, point2_longitude)

    map_center = [32.1667, -8.5167]
    map_youssoufia = folium.Map(location=map_center, zoom_start=12)

    folium.Marker(location=[point1_latitude, point1_longitude], popup=point1_name).add_to(map_youssoufia)
    folium.Marker(location=[point2_latitude, point2_longitude], popup=point2_name).add_to(map_youssoufia)

    folium.PolyLine(locations=[[point1_latitude, point1_longitude], [point2_latitude, point2_longitude]], color='blue').add_to(map_youssoufia)
    folium.Marker(location=[(point1_latitude + point2_latitude) / 2, (point1_longitude + point2_longitude) / 2], popup=f"Distance: {distance_entre_points:.2f} km", icon=folium.Icon(icon='cloud')).add_to(map_youssoufia)

    map_youssoufia.save('carte_youssoufia.html')

display_distance_on_map("Youcode", "Monte Carlo Coffeeshop", data_json)
